In [2]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import pydicom
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [4]:
# Constants
IMAGE_SIZE = (224, 224)  # MobileNet requires input images of size (224, 224)
BATCH_SIZE = 32

In [ ]:
# Functions
# Load and preprocess DICOM image
def load_dicom_image(dicom_path):
    # Load the DICOM file
    dicom = pydicom.dcmread(dicom_path)
    # Convert DICOM pixel array to a NumPy array
    image = dicom.pixel_array
    # Normalize the image to range [0, 1]
    image = image.astype(np.float32) / np.max(image)
    # Convert grayscale images to RGB by stacking
    if len(image.shape) == 2:
        image = np.stack((image,)*3, axis=-1)
    # Resize the image to match MobileNet's input size
    image = tf.image.resize(image, IMAGE_SIZE)
    return image

# Function to process dataset
def preprocess_dicom_dataset(dicom_paths, labels):
    # Create a TensorFlow dataset from the lists of dicom paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((dicom_paths, labels))

    # Define a function to load and preprocess each DICOM image
    def load_and_preprocess(dicom_path, label):
        image = tf.py_function(func=load_dicom_image, inp=[dicom_path], Tout=tf.float32)
        image.set_shape([*IMAGE_SIZE, 3])  # Set the shape for TensorFlow to know the static dimensions
        return image, label

    # Apply the loading and preprocessing function
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle, batch, and prefetch the dataset
    dataset = dataset.shuffle(buffer_size=len(dicom_paths)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:

# Sample DICOM paths and labels (these would be your actual DICOM files and labels)
dicom_paths = ["path/to/dicom1.dcm", "path/to/dicom2.dcm", "path/to/dicom3.dcm"]
labels = [0, 1, 0]  # Example labels

# Create the dataset
dataset = preprocess_dicom_dataset(dicom_paths, labels)

# Load MobileNet with pretrained weights
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))

# Add custom layers on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the DICOM dataset
model.fit(dataset, epochs=10)
